# TD 1 - Révolutions : Explorations du terme à partir de Wikipedia

## 1. Extraire le corpus

Importons tout d'abord les bibliothèques nécessaires à l'extraction des données sur Wikipedia.

In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import re
import requests

Ensuite, connectons-nous sur la page "Révolution (homonymie)" pour tenter de récupérer une liste des diverses expressions différentes du terme "révolution" dans l'encyclopédie.

In [2]:
url = "https://fr.wikipedia.org/wiki/R%C3%A9volution_(homonymie)"
base_url = "https://fr.wikipedia.org"
r = requests.get(url)
soup = BeautifulSoup(r.text, "html5lib")
# Let's get the body of the article
body = soup.find("div", attrs={"class", "mw-parser-output"})
# Get the list of h2
h3 = body.find_all("h3")

Construisons maintenant le modèle de notre futur tableau de données.

In [3]:
links = {"liens": [],
        "nom_page": []}

A présent, passons à la récupération des données filtrées. On ne récupère que le titre et le lien vers chacun des pages rencontrées dans la page Wikipedia. Nous ferons un peu de nettoyage plus tard.

In [4]:
hrefs = body.find_all("a")
regex = re.compile("[Rr][ée]volution[s]?")

for l in hrefs:
    try:
        if re.match(regex, l["title"]):
            links["liens"].append(base_url + l["href"])
            links["nom_page"].append(l["title"])
    except:
        pass

A partir des données récupérées, on peut créer un tableau qui contiendra le nom des pages et les liens. Grâce à cela, on pourra récupérer le text intégral des pages Wikipedia qui nous intéressent.

In [5]:
df = pd.DataFrame(links)

Ajoutons maintenant le texte intégral des pages web qui nous intéressent dans notre tableau.

In [6]:
full_text = []

In [7]:
for link in tqdm(df.iterrows()):
    r = requests.get(link[1]["liens"])
    soup = BeautifulSoup(r.text, "html5lib")
    try:
        body = soup.find("div", attrs={"class", "mw-content-container"})
        full_text.append(body.get_text())
    except AttributeError:
        full_text.append(soup.get_text())

76it [00:35,  2.17it/s]


In [8]:
df["full_text"] = full_text

Nous pouvons maintenir exporter le résultat dans un tableau CSV.

In [9]:
df.to_csv("revolution.csv", sep=";", encoding="utf-8")

Nous pouvons créer une fonction pour nettoyer les corpus.

In [10]:
df.head()

,liens,nom_page,full_text
0,https://fr.wikipedia.org/wiki/R%C3%A9volution_...,Révolution industrielle,\n\n\n\t\n\t\n\t\n\t\n\tRévolution industriell...
1,https://fr.wikipedia.org/wiki/R%C3%A9volution_...,Révolution (politique et sociale),\n\n\n\t\n\t\n\t\n\t\n\tRévolution\n\t\n\t\tUn...
2,https://fr.wikipedia.org/wiki/R%C3%A9volution_...,Révolution sociale,\n\n\n\t\n\t\n\t\n\t\n\tRévolution sociale\n\t...
3,https://fr.wikipedia.org/wiki/R%C3%A9volution_...,Révolution néolithique,\n\n\n\t\n\t\n\t\n\t\n\tRévolution néolithique...
4,https://fr.wikipedia.org/wiki/R%C3%A9volution_...,Révolution agricole,\n\n\n\t\n\t\n\t\n\t\n\tRévolution agricole\n\...


## 2. Nettoyage du corpus

Nous récupérons un corpus de texte brut qu'il nous faut nettoyer avant de l'utiliser.

Premièrement, retirons tous les "grands" caractères d'espacement (`\n`, `\t`).

In [18]:
def remove_widespaces(text):
    regex = re.compile(r"\n|\t")
    text = re.sub(regex, " ", text)
    return text

Ensuite, retirons les espaces non sécables.

In [19]:
def remove_nonbreaking_spaces(text):
    regex = re.compile(r"\xa0")
    text = re.sub(regex, " ", text)
    return text

Retirons maintenant les notes inscrites entre crochets.

In [20]:
def remove_notes(text):
    regex = re.compile(r"\[*?\]")
    text = re.sub(regex, " ", text)
    return text

Ensuite, retirons des éléments de formatage de Wikipedia.

In [21]:
def remove_wikicode(text):
    regex = re.compile(r"\[modifier | modifier le code")
    text = re.sub(regex, " ", text)
    return text

Enfin, retirons les notes numérotées mal formatées.

In [22]:
def remove_num_notes(text):
    regex = re.compile(r"\[\d+")
    text = re.sub(regex, " ", text)
    return text

Retirons également les barres obliques.

In [23]:
def remove_vertical(text):
    regex = re.compile(r"\|")
    text = re.sub(regex, " ", text)
    return text

Enfin, retirons les flèches verticales utilisées dans les notes de bas de page.

In [24]:
def remove_arrow(text):
    regex = re.compile(r"↑")
    text = re.sub(regex, " ", text)
    return text

Appliquons toutes les fonctions de nettoyage sur le corpus.

In [27]:
for index, row in df.iterrows():
    new_row = remove_arrow(remove_vertical(remove_num_notes(remove_wikicode(remove_notes(remove_nonbreaking_spaces(remove_widespaces(row["full_text"])))))))
    row["full_text"] = new_row

Exportons à présent le corpus nettoyé.

In [29]:
df.to_csv("revolution_clean.csv", sep=";", encoding="utf-8")

## 3. Préparation du corpus pour Iramuteq

In [10]:
import pandas as pd

In [11]:
df_clean = pd.read_csv("revolution_clean.csv", sep=";", encoding="utf-8")

In [12]:
title = []
full_text = []

for index, row in df_clean.iterrows():
    title_nospace = row["nom_page"].replace(" ", "-")
    title.append(title_nospace)
    full_text.append(row["full_text"])


with open("corpus_revolution_iramuteq.txt", "w", encoding="utf-8") as file:
    for t, f in zip(title, full_text):
        
        file.write(f"**** *titre_{t} \n\n {f} \n\n")